# Static Visualization Draft

Description of project

Data Sources

Narrative, questions

In [ ]:
import pandas as pd
import altair as alt

In [12]:
state_lvl_df = pd.read_excel('state_level_est.xlsx')

In [13]:
census_mhi_df = pd.read_csv('census_mhi_2022.csv')
census_mhi_df = census_mhi_df.rename(columns={"State":"STATE_NAME", "MHI":"MHI"})

In [20]:
big_22_ndcp_df = pd.read_excel('2022ndcp.xlsx')

## Graph Set One

Grouped bar charts looking at childcare cost by developmental cohort. I look at both national level and Illinois level data. 

Currently I think the number of subgraphs is overwhelming, and I'm wondering if there's a better graph type or way that I could subsection the current graphs to be more digestible. 

Another area where I'd appreciate feedback (in general, but particularly here) is color. Normally nominal data will use categorical color schemes, but you could also consider the age cohorts to be ordered data, in which case, perhaps a sequential color scheme would be more appropriate. Thoughts?

In [14]:
#state avg cost data
def state_avg_costs_df(df):
    #2022 center data
    #eliminate states that don't have data for all four age groups
    #eliminate states where less than 90% of kids are living in counties w data
    clean_df = df[(df['STUDYYEAR'] == 2022) & (df['MEDIAN_INFANT_PRICE'].notna()) & (df['TYPE'] == 'Center') & (df['PCT_VALID_INFANT'] >= .9) & (df['PCT_VALID_TODDLER'] >= .9) & (df['PCT_VALID_PRESCHOOL'] >= .9) & (df['PCT_VALID_SCHOOLAGE'] >= .9)]
    select_columns_df = clean_df[['STATE_NAME', 'MEDIAN_INFANT_PRICE', 'MEDIAN_TODDLER_PRICE', 'MEDIAN_PRESCHOOL_PRICE', 'MEDIAN_SCHOOLAGE_PRICE']]
    
    return select_columns_df

In [15]:
def grouped_bar_df(df, census_df):
    df = df.melt(
        id_vars=['STATE_NAME'],
        value_vars=['MEDIAN_INFANT_PRICE', 'MEDIAN_TODDLER_PRICE', 'MEDIAN_PRESCHOOL_PRICE', 'MEDIAN_SCHOOLAGE_PRICE'],
        var_name='age',
        value_name='price'
    )
    df['age'] = df['age'].str.replace('_PRICE', '').str.replace('MEDIAN_', '').str.lower()
    merged = pd.merge(df, census_df, "left", "STATE_NAME")
    merged["adjusted price"] = merged['price'] * 52 / merged['MHI']

    return merged

In [243]:
def groupedbar(df, title, norm, level_name):
    age_order = ['infant', 'toddler', 'preschool', 'schoolage']
    alt.Chart(df).mark_bar().encode(
        alt.X('age:N', sort=age_order).title('Age Group'),
        alt.Y(f'{norm}:Q').title('Weekly Cost of Childcare for One Child'),
        alt.Color('age:N').scale(scheme='turbo'),
        column=f'{level_name}:N'
    ).properties(title=title).show()

### National, State Level, *Not* Normalized

In [244]:
state_avgs = state_avg_costs_df(state_lvl_df)
grouped_df = grouped_bar_df(state_avgs, census_mhi_df)
groupedbar(grouped_df, "2022 Weekly Childcare Cost by State", "price", "STATE_NAME")

alt.Chart(...)

### National, State Level, Normalized

In [245]:
groupedbar(grouped_df, "2022 Childcare Costs Normalized by State Median Family Income",
                    "adjusted price", "STATE_NAME")

alt.Chart(...)

In [91]:
def clean_il_df(df):
    df = df[['STATE_ABBREVIATION', 'COUNTY_NAME', 'COUNTY_FIPS_CODE', 'STUDYYEAR', 'MHI', 'MCSA', 'MCINFANT', 'MCTODDLER', 'MCPRESCHOOL']]
    il_df = df[(df['STATE_ABBREVIATION'] == 'IL') & (df['STUDYYEAR'] == 2022)]
    il_df = il_df[(il_df['MCSA'].notna()) & (il_df['MCPRESCHOOL'].notna()) & (il_df['MCTODDLER'].notna()) & (il_df['MCINFANT'].notna())]

    return il_df

In [ ]:
def grouped_il_bar_df(df):
    melt_df = df.melt(
        id_vars=['COUNTY_NAME'],
        value_vars=['MCSA', 'MCINFANT', 'MCTODDLER', 'MCPRESCHOOL'],
        var_name='age',
        value_name='price'
    )
    melt_df['age'] = melt_df['age'].str.replace('MC', '').str.lower()
    melt_df['age'] = melt_df['age'].str.replace('sa', 'schoolage')
    merged = pd.merge(melt_df, df[['COUNTY_NAME', 'MHI']], "left", "COUNTY_NAME")
    merged["adjusted price"] = merged['price'] * 52 / merged['MHI']

    return merged

In [247]:
il_df = clean_il_df(big_22_ndcp_df)
il_df = grouped_il_bar_df(il_df)

### Illinois, County-Level, Normalized

In [248]:
groupedbar(il_df, "2022 Childcare Cost Normalized by County Median Household Income", "adjusted price", "COUNTY_NAME")

alt.Chart(...)

Income analysis for excluded states and counties

## Graph Set Two

This set of graphs are a collection of scatterplots that look at childcare cost burden for school age children (easily the least expensive, across the board) ploted against cost burden for care services for younger children, which is more specialized.

Once again, I did national and Illinois level graphs. I started just plotting schoolage against infantcare, but then I wondered if there would be noticeable differences between schoolage and the other two categories and did tricolor plots. I'm not super wowwed by any of them, so I would appreciate feedback on better graph types or whether this kind of graph should be included in my final project.

In [97]:
state_costs_df = state_avg_costs_df(state_lvl_df)
state_costs_df = pd.merge(state_costs_df, census_mhi_df, "left", "STATE_NAME")
state_costs_df['adj_infant'] = (state_costs_df["MEDIAN_INFANT_PRICE"] * 52) / state_costs_df['MHI']
state_costs_df['adj_toddler'] = (state_costs_df["MEDIAN_TODDLER_PRICE"] * 52) / state_costs_df['MHI']
state_costs_df['adj_preschool'] = (state_costs_df["MEDIAN_PRESCHOOL_PRICE"] * 52) / state_costs_df['MHI']
state_costs_df['adj_schoolage'] = (state_costs_df["MEDIAN_SCHOOLAGE_PRICE"] * 52) / state_costs_df['MHI']

In [249]:
def schoolage_v_infant_scatter(df, title):
    alt.Chart(df).mark_circle().encode(
        alt.X('adj_schoolage').scale(domain=(0, .3)).axis(format='%').title("Normalized Schoolage Childcare Cost"),
        alt.Y('adj_infant').scale(domain=(0, .3)).axis(format='%').title("Normalized Infantcare Cost"),
    ).properties(title=title).show()

### National, Infantcare vs Schoolage Care

In [250]:
schoolage_v_infant_scatter(state_costs_df, "School Age Childcare vs Infantcare Burden by State")

alt.Chart(...)

In [251]:
def schoolage_multi_scatter(df, title):
    chart = alt.Chart(df)
    
    infant = chart.mark_circle(color="blue", opacity=.5).encode(
        alt.X('adj_schoolage').scale(domain=(0, .3)).axis(format='%').title("Normalized Schoolage Childcare Cost"),
        alt.Y('adj_infant').scale(domain=(0, .3)).axis(format='%').title("Under-Schoolage Childcare Normalized Cost"),
    )

    toddler = chart.mark_circle(color="red", opacity=.5).encode(
        alt.X('adj_schoolage').scale(domain=(0, .3)).axis(format='%').title("Normalized Schoolage Childcare Cost"),
        alt.Y('adj_toddler').scale(domain=(0, .3)).axis(format='%').title("Under-Schoolage Childcare Normalized Cost"),
    )

    preschool = chart.mark_circle(color="yellow", opacity=.5).encode(
        alt.X('adj_schoolage').scale(domain=(0, .3)).axis(format='%').title("Normalized Schoolage Childcare Cost"),
        alt.Y('adj_preschool').scale(domain=(0, .3)).axis(format='%').title("Under-Schoolage Childcare Normalized Cost"),
    )

    layered = infant + toddler + preschool

    layered.properties(title=title).show()

### National, All Younger Care vs Schoolage Care

In [252]:
schoolage_multi_scatter(state_costs_df, "School Age Childcare vs Under 6 Childcare Burden by State")

alt.LayerChart(...)

In [254]:
il_df = clean_il_df(big_22_ndcp_df)
il_df['adj_infant'] = (il_df["MCINFANT"] * 52) / il_df['MHI']
il_df['adj_toddler'] = (il_df["MCTODDLER"] * 52) / il_df['MHI']
il_df['adj_preschool'] = (il_df["MCPRESCHOOL"] * 52) / il_df['MHI']
il_df['adj_schoolage'] = (il_df["MCSA"] * 52) / il_df['MHI']

### Illinois, Infantcare vs Schoolage Care

In [255]:
schoolage_v_infant_scatter(il_df, "Illinois School Age Childcare vs Infantcare Burden by County")

alt.Chart(...)

### Illinois, All Younger Care vs Schoolage Care

In [256]:
schoolage_multi_scatter(il_df, "Illinois School Age Childcare vs Under 6 Childcare Burden by County")

alt.LayerChart(...)

## Graph Set Three

This graph is a scatterplot that considers cost burden of childcare by cohort (School Age vs Preschool) and female labor participation rate for mothers with children in those respective categories.

I included regression lines, but the presence of outliers makes me skeptical of their quality, and the data doesn't seem to produce visible trends for either cluster. Would be curious to try this out with other states and see if that's still the case.

In [257]:
def get_il_w_normcost(df):
    il_df = clean_il_df(df)
    il_df['adj_infant'] = (il_df["MCINFANT"] * 52) / il_df['MHI']
    il_df['adj_toddler'] = (il_df["MCTODDLER"] * 52) / il_df['MHI']
    il_df['adj_preschool'] = (il_df["MCPRESCHOOL"] * 52) / il_df['MHI']
    il_df['adj_schoolage'] = (il_df["MCSA"] * 52) / il_df['MHI']

    return il_df

In [258]:
def get_flpr_df(df):

    female_labor_df = df[(df['STATE_ABBREVIATION'] == 'IL'
                                    ) & (df['STUDYYEAR'] == 2022)]
    female_labor_df = female_labor_df[(female_labor_df['MCSA'].notna()
                                    ) & (female_labor_df['MCPRESCHOOL'].notna()
                                            ) & (female_labor_df['MCTODDLER'].notna()
                                                ) & (female_labor_df['MCINFANT'].notna())]

    flpr_var = ['COUNTY_NAME', 'FLFPR_20to64', 'FLFPR_20to64_UNDER6', 'FLFPR_20to64_6to17',
                'FLFPR_20to64_UNDER6_6to17']

    female_labor_df = female_labor_df[flpr_var]

    flpr_var.pop(0)

    for col in flpr_var:
        female_labor_df[col] = female_labor_df[col] / 100

    return female_labor_df

In [259]:
il_df = get_il_w_normcost(big_22_ndcp_df)
female_labor_df = get_flpr_df(big_22_ndcp_df)
il_df = pd.merge(il_df, female_labor_df, 'left', 'COUNTY_NAME')

In [261]:
def labor_burden_scatter(df, title):
    chart = alt.Chart(df)

    preschool = chart.mark_circle(color='teal', opacity=.5).encode(
        alt.X('adj_preschool').axis(format='%').title("Normalized Schoolage Childcare Cost"),
        alt.Y('FLFPR_20to64_UNDER6:Q').axis(format='%').title("Maternal Labor Force Participation"),
    )

    preschool = preschool + preschool.transform_regression('adj_preschool', 'FLFPR_20to64_UNDER6').mark_line(color="teal")

    schoolage = chart.mark_circle(color='orange', opacity=.5).encode(
        alt.X('adj_schoolage').axis(format='%').title("Normalized Schoolage Childcare Cost"),
        alt.Y('FLFPR_20to64_6to17:Q').axis(format='%').title("Maternal Labor Force Participation"),
    )

    schoolage = schoolage + schoolage.transform_regression('adj_schoolage', 'FLFPR_20to64_6to17').mark_line(color="orange")

    layered = preschool + schoolage

    layered.properties(title=title).show()

In [262]:
labor_burden_scatter(il_df, "IL FLPR for Mothers of School Age and Under 6 Children")

alt.LayerChart(...)

## Graph Set Four

For this group I have made heat maps that look at childcare burden for preschool age children and female labor force participation. The first heatmap looks exclusively at female labor force participation for mothers with children under the age of 6. The second looks at the general female labor force participation rate (ages 20-64) to see if it's remarkably different. We see that it is in fact different, the rate appears to be lower for women who *don't* have very small children, which runs contrary to intuition.

In [223]:
def young_flpr_df(df):
    yflpr_df = df[(df['STUDYYEAR'] == 2022)]
    select_columns = ['COUNTY_NAME', 'MHI', 'MCINFANT', 'MCTODDLER', 'MCPRESCHOOL', 'MCSA', 'FLFPR_20to64', 'FLFPR_20to64_UNDER6', 'FLFPR_20to64_6to17']
    yflpr_df = yflpr_df[select_columns]
    #'FLFPR_20to64_UNDER6_6to17'
    pct_tidy = ['FLFPR_20to64', 'FLFPR_20to64_UNDER6', 'FLFPR_20to64_6to17']

    for col in pct_tidy:
        yflpr_df[col] = yflpr_df[col] / 100

    adj_val = ['MCINFANT', 'MCTODDLER', 'MCPRESCHOOL', 'MCSA']

    for col in adj_val:
        nice_name = col[2:].lower()
        yflpr_df[f'adj_{nice_name}'] = (yflpr_df[col] * 52) / yflpr_df['MHI']

    return yflpr_df

In [224]:
yflpr_df = young_flpr_df(big_22_ndcp_df)

In [267]:
def flpr_heatmap(df, young_mothers, title):
    if young_mothers:
        y_var = 'FLFPR_20to64_UNDER6'
    else:
        y_var = 'FLFPR_20to64'
    alt.Chart(df).mark_rect().encode(
        alt.X('adj_preschool:Q').bin(step=.01).title("Preschool Age Childcare Cost Burden Normalized by MHI"),
        alt.Y(f'{y_var}:Q').bin(step=.05).title("Maternal Labor Force Participation Rate (Child Under Age Six)"),
        alt.Color('count():Q').scale(scheme='greenblue').legend(title='Number of Counties')
    ).properties(title=title).show()

In [268]:
flpr_heatmap(yflpr_df, True, "2022 FLPR by County for Mothers of Children Under Six")

alt.Chart(...)

In [269]:
flpr_heatmap(yflpr_df, False, "2022 FLPR by County")

alt.Chart(...)

## Graph Set Five

I wanted to do some kind of line graph analyzing trends in childcare prices over time. For the faceted graph I made, I focused only on prices for school age children, and I only did states that have complete data for the full range of years (2008-2022). Doing all the states side by side was too visually overwhelming, so I opted to group them by region. 

The way I've done it here uses way too many colors, and it's too confusing to differentiate states. I'm wondering if it's possible to do something analogous to a multi-panel scatter plot here.

In [331]:
def clear_incomplete_states(df):
    prices = ['MEDIAN_SCHOOLAGE_PRICE', 'MEDIAN_PRESCHOOL_PRICE', 'MEDIAN_TODDLER_PRICE', 'MEDIAN_INFANT_PRICE']
    df_clean = df.dropna(subset=prices)
    years_per_state = tt_clean.groupby('STATE_NAME')['STUDYYEAR'].nunique()
    expected_years = df['STUDYYEAR'].nunique()
    states_with_all_years = years_per_state[years_per_state == expected_years].index
    df_clean = df_clean[df_clean['STATE_NAME'].isin(states_with_all_years)]

    return df_clean

In [332]:
def add_region(df):
    SOUTHEAST = ['Alabama', 'Florida', 'Kentucky', 'Louisiana', 'Tennessee', 'West Virginia']
    MIDWEST = ['Illinois', 'Kansas', 'Michigan', 'Minnesota', 'North Dakota', 'South Dakota', 'Wisconsin']
    WEST = ['Arizona', 'Oklahoma', 'Oregon', 'Utah', 'Washington']
    NORTHEAST = ['Connecticut', 'Delaware', 'Massachusetts', 'New Jersey']

    region_map = {}
    for state in NORTHEAST:
        region_map[state] = 'Northeast'
    for state in MIDWEST:
        region_map[state] = 'Midwest'
    for state in SOUTHEAST:
        region_map[state] = 'South'
    for state in WEST:
        region_map[state] = 'West'

    df['REGION'] = df['STATE_NAME'].map(region_map)

    return df

In [333]:
def time_trends_df(df):
    tt_df = df[df['TYPE'] == 'Center']
    tt_df = tt_df[['STATE_NAME', 'TYPE', 'STUDYYEAR', 'MEDIAN_SCHOOLAGE_PRICE', 'MEDIAN_PRESCHOOL_PRICE', 'MEDIAN_TODDLER_PRICE', 'MEDIAN_INFANT_PRICE']]
    tt_df = clear_incomplete_states(tt_df)
    tt_df = add_region(tt_df)

    return tt_df

In [334]:
tt_df = time_trends_df(state_lvl_df)

In [343]:
def time_trends(df, title, cohort):
    alt.Chart(df).mark_line().encode(
        alt.X('STUDYYEAR:O'),
        alt.Y(f'{cohort}:Q'),
        alt.Color('STATE_NAME:N')
    ).facet(facet='REGION:N', columns=2).properties(title=title).show()

In [344]:
time_trends(tt_df, "School Age Childcare Costs 2008-2022 by State", 'MEDIAN_SCHOOLAGE_PRICE')

alt.FacetChart(...)